# 2021/10/19
Read the best parameter set in the pkl model

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
import pandas as pd
import numpy as np
import sys, os

In [4]:
import joblib
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/2021-10-08'
pkl = 'rf.pkl'
rf_model = joblib.load(os.path.join(input_path, pkl))
rf_model

Pipeline(steps=[('smote', SMOTE(random_state=42)),
                ('rfr',
                 RandomForestClassifier(max_depth=5, min_samples_leaf=2,
                                        n_estimators=80, random_state=42))])

In [ ]:
## Repeat the results

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report


#input_path='/pod/2/li-lab/Ziwei/Nanopore/results/megalodon2.3.4_guppy5.0.14'
#df=pd.read_csv(os.path.join(input_path, 'total.Megalodon.per_read.prob.bed.gz'),compression='gzip', sep='\t')

df=pd.read_csv('/pod/2/li-lab/Ziwei/Nanopore/daily/test/total.test.bed.gz',compression='gzip', sep='\t')
print("Data is loading!")

#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

X = df_feature
y = df_class

#Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)
print("Spliting is done!")

Data is loading!
Spliting is done!


In [7]:
rf_result = rf_model.fit(X_train, y_train)

y_true, y_pred = y_test, rf_model.predict(X_test)

## Load the training result on cv

In [18]:
from sklearn.model_selection import cross_val_score

stratified_kfold = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)

cv_score = cross_val_score(rf_model, X_train, y_train, 
                           cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
print(cv_score,cv_score.mean())

[0.90711218 0.89945412 0.91220319] 0.9062564948951842


## Load the testing result

In [13]:
test_score = rf_result.score(X_test, y_test)
print(type(rf_result))
print('Test score:', test_score)

<class 'imblearn.pipeline.Pipeline'>
Test score: 0.882


### Save the classification report of testing result

In [21]:
## Save the classification report
target_class = ['5C', '5mC', '5hmC']
report = classification_report(y_test, y_pred, target_names=target_class, output_dict=True)
report = pd.DataFrame(report).transpose()
report

,precision,recall,f1-score,support
5C,0.929293,0.920,0.924623,200.000
5mC,0.957219,0.895,0.925065,200.000
5hmC,0.678261,0.780,0.725581,100.000
accuracy,0.882000,0.882,0.882000,0.882
macro avg,0.854924,0.865,0.858423,500.000
weighted avg,0.890257,0.882,0.884991,500.000


### Save the confusion matrix of testing result

In [23]:
##Save the confusion matrix   
plot_cm(y_test, y_pred, os.path.join(output_path, cm_png), labels=None)

NameError: name 'plot_cm' is not defined